<a href="https://colab.research.google.com/github/nezhalahnech/game-of-life/blob/master/roadmark_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()   # اختاري ملف .zip من جهازك


In [ ]:
!unzip /content/ROADMARK.zip -d /content/ROADMARK

In [ ]:
import os

for root, dirs, files in os.walk("/content/ROADMARK"):
    level = root.replace("/content/ROADMARK", "").count(os.sep)
    indent = " " * (4 * level)
    print(f"{indent}{os.path.basename(root)}/")
    subindent = " " * (4 * (level + 1))
    for f in files[:5]:  # نوري غير أول 5 ملفات باش مايطولش
        print(f"{subindent}{f}")


ROADMARK/
    README.roboflow.txt
    README.dataset.txt
    data.yaml
    test/
        images/
            618d_jpg.rf.1cc699d237b8fe7a55f69ad4e2d85754.jpg
            868c_jpg.rf.a5fb446ed0d34d9fc99e961b091eccb7.jpg
            218b_jpg.rf.19781ebb54ac5952a187ac13edb45960.jpg
            653b_jpg.rf.5d63b76d67abaed121e50f56dcdeff12.jpg
            878a_jpg.rf.54bf46cb7c056766c700b08dc8acbacd.jpg
        labels/
            307a_jpg.rf.2a6687571cfdea1f1d17dba193f387ff.txt
            235d_jpg.rf.aad6f7e785dc2398583384a18a52a7a7.txt
            723_jpg.rf.3e56ec6db5e7d414d98fb24ea4277943.txt
            201a_jpg.rf.b34d8dc165f51d7447ee655348fba867.txt
            575_jpg.rf.29dbd2a5323d942449dbfad5b91ae098.txt
    valid/
        labels.cache
        images/
            263c_jpg.rf.c112ccc3168c2d6295e936cce5ccad97.jpg
            380b_jpg.rf.20f45b4c27a98ef38e08bd7231da4cbb.jpg
            776a_jpg.rf.8e93e8495d0b407d70e2eca35666e179.jpg
            176b_jpg.rf.333561e58ac1aa88d799ea6c

In [ ]:
# install ultralytics (YOLOv8)
!pip install ultralytics

In [ ]:
import os

def clean_labels(labels_path, remove_ids=["0","1"]):  # hna 7t IDs dyal خطوط الطريق
    for filename in os.listdir(labels_path):
        if filename.endswith(".txt"):
            filepath = os.path.join(labels_path, filename)
            with open(filepath, "r") as f:
                lines = f.readlines()
            new_lines = [line for line in lines if line.split()[0] not in remove_ids]
            with open(filepath, "w") as f:
                f.writelines(new_lines)

# Apply 3la train/valid/test
clean_labels("/content/ROADMARK/train/labels")
clean_labels("/content/ROADMARK/valid/labels")
clean_labels("/content/ROADMARK/test/labels")


In [ ]:
from ultralytics import YOLO

# Load model (yolov8n = nano, tu peux changer)
model = YOLO("yolov8n.pt")

# Start training
model.train(
    data="/content/ROADMARK/data.yaml",  # path dyal dataset config
    epochs=50,
    imgsz=640,
    batch=16,
    name="roadmark_cleaned"
)


Ultralytics 8.3.198 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/ROADMARK/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=roadmark_cleaned4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots

In [ ]:
from ultralytics import YOLO

# Load model (yolov8n = nano, tu peux changer)
model = YOLO("yolov8n.pt")

# Start training
model.train(
    data="/content/ROADMARK/data.yaml",  # path dyal dataset config
    epochs=50,
    imgsz=640,
    batch=16,
    name="roadmark_cleaned"
)


In [ ]:
from ultralytics import YOLO

# Load trained model
model = YOLO("/content/runs/detect/roadmark_cleaned3/weights/best.pt")

# Evaluate on test set
metrics = model.val()
print(metrics)

In [ ]:
import yaml

dataset_yaml = {
    "path": "/content/ROADMARK",
    "train": "train/images",
    "val": "valid/images",
    "test": "test/images",
    "names": ["arrow", "crosswalk", "stop", "bus"]
}

with open("roadmark.yaml", "w") as f:
    yaml.dump(dataset_yaml, f, default_flow_style=False)

print(open("roadmark.yaml").read())

In [ ]:
from ultralytics import YOLO

# load pretrained YOLOv8 model (small)
model = YOLO("yolov8s.pt")

# train on the cleaned dataset
model.train(data="roadmark.yaml", epochs=50, imgsz=640, batch=16)

In [ ]:
# Evaluate on validation/test set
metrics = model.val()

# Print metrics
print(metrics)